In [1]:
import pandas as pd              
import matplotlib.pyplot as plt   
import seaborn as sns     
import numpy as np                 
%matplotlib inline

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

events = pd.read_csv('holidays_events.csv')
oil = pd.read_csv('oil.csv')
stores = pd.read_csv('stores.csv')
transaction = pd.read_csv('transactions.csv')

In [3]:
train.head(3)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0


In [4]:
test.tail(3)

,id,date,store_nbr,family,onpromotion
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9
28511,3029399,2017-08-31,9,SEAFOOD,0


In [5]:
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
events["date"] = pd.to_datetime(events.date)
oil["date"] = pd.to_datetime(oil.date)
transaction["date"] = pd.to_datetime(transaction.date)

In [6]:
train.head(3)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0


In [7]:
dates = pd.bdate_range("2013-01-01","2017-08-31",freq='D')
oil_date = pd.DataFrame(data=dates,columns=['date'])
oil_date = pd.merge(left=oil_date,right=oil,how='left',on='date')

In [8]:
oil_date['dcoilwtico_interpolate'] = oil_date['dcoilwtico'].interpolate()
oil_date

,date,dcoilwtico,dcoilwtico_interpolate
0,2013-01-01,NaN,NaN
1,2013-01-02,93.14,93.140000
2,2013-01-03,92.97,92.970000
3,2013-01-04,93.12,93.120000
4,2013-01-05,NaN,93.146667
...,...,...,...
1699,2017-08-27,NaN,46.816667
1700,2017-08-28,46.40,46.400000
1701,2017-08-29,46.46,46.460000
1702,2017-08-30,45.96,45.960000


In [9]:
oil_date = oil_date.drop('dcoilwtico', axis=1)
oil_date.at[0,'dcoilwtico_interpolate'] = oil_date.at[1,'dcoilwtico_interpolate']
oil_date

,date,dcoilwtico_interpolate
0,2013-01-01,93.140000
1,2013-01-02,93.140000
2,2013-01-03,92.970000
3,2013-01-04,93.120000
4,2013-01-05,93.146667
...,...,...
1699,2017-08-27,46.816667
1700,2017-08-28,46.400000
1701,2017-08-29,46.460000
1702,2017-08-30,45.960000


In [10]:
train_all = pd.merge(train,events,how='left',on='date')
train_all = pd.merge(train_all,oil_date,how='left',on='date')
#train_all = pd.merge(train_all,transaction,how='left',on=['date','store_nbr'])
train_all = pd.merge(train_all,stores,how='left',on='store_nbr')

In [11]:
test_all = pd.merge(test,events,how='left',on='date')
test_all = pd.merge(test_all,oil_date,how='left',on='date')
#test_all = pd.merge(test_all,transaction,how='left',on=['date','store_nbr'])
test_all = pd.merge(test_all,stores,how='left',on='store_nbr')

In [12]:
train_all.head(3)

,id,date,store_nbr,family,sales,onpromotion,type_x,locale,locale_name,description,transferred,dcoilwtico_interpolate,city,state,type_y,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,93.14,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,93.14,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,93.14,Quito,Pichincha,D,13


In [13]:
test_all.head(3)

,id,date,store_nbr,family,onpromotion,type_x,locale,locale_name,description,transferred,dcoilwtico_interpolate,city,state,type_y,cluster
0,3000888,2017-08-16,1,AUTOMOTIVE,0,NaN,NaN,NaN,NaN,NaN,46.8,Quito,Pichincha,D,13
1,3000889,2017-08-16,1,BABY CARE,0,NaN,NaN,NaN,NaN,NaN,46.8,Quito,Pichincha,D,13
2,3000890,2017-08-16,1,BEAUTY,2,NaN,NaN,NaN,NaN,NaN,46.8,Quito,Pichincha,D,13


In [14]:
train_all.dtypes

id                                 int64
date                      datetime64[ns]
store_nbr                          int64
family                            object
sales                            float64
onpromotion                        int64
type_x                            object
locale                            object
locale_name                       object
description                       object
transferred                       object
dcoilwtico_interpolate           float64
city                              object
state                             object
type_y                            object
cluster                            int64
dtype: object

In [15]:
test_all.dtypes

id                                 int64
date                      datetime64[ns]
store_nbr                          int64
family                            object
onpromotion                        int64
type_x                            object
locale                            object
locale_name                       object
description                       object
transferred                       object
dcoilwtico_interpolate           float64
city                              object
state                             object
type_y                            object
cluster                            int64
dtype: object

In [16]:
train_all.isnull().sum()

id                              0
date                            0
store_nbr                       0
family                          0
sales                           0
onpromotion                     0
type_x                    2551824
locale                    2551824
locale_name               2551824
description               2551824
transferred               2551824
dcoilwtico_interpolate          0
city                            0
state                           0
type_y                          0
cluster                         0
dtype: int64

In [17]:
test_all.isnull().sum()

id                            0
date                          0
store_nbr                     0
family                        0
onpromotion                   0
type_x                    26730
locale                    26730
locale_name               26730
description               26730
transferred               26730
dcoilwtico_interpolate        0
city                          0
state                         0
type_y                        0
cluster                       0
dtype: int64

In [18]:
train_all['type_x'] = train_all['type_x'].fillna("Normal")
train_all['locale'] = train_all['locale'].fillna("Normal")
train_all['locale_name'] = train_all['locale_name'].fillna("Normal")
train_all['description'] = train_all['description'].fillna("Normal")
train_all['transferred'] = train_all['transferred'].fillna(False)

In [19]:
test_all['type_x'] = test_all['type_x'].fillna("Normal")
test_all['locale'] = test_all['locale'].fillna("Normal")
test_all['locale_name'] = test_all['locale_name'].fillna("Normal")
test_all['description'] = test_all['description'].fillna("Normal")
test_all['transferred'] = test_all['transferred'].fillna(False)

In [20]:
train_all.isnull().sum()

id                        0
date                      0
store_nbr                 0
family                    0
sales                     0
onpromotion               0
type_x                    0
locale                    0
locale_name               0
description               0
transferred               0
dcoilwtico_interpolate    0
city                      0
state                     0
type_y                    0
cluster                   0
dtype: int64

In [21]:
test_all.isnull().sum()

id                        0
date                      0
store_nbr                 0
family                    0
onpromotion               0
type_x                    0
locale                    0
locale_name               0
description               0
transferred               0
dcoilwtico_interpolate    0
city                      0
state                     0
type_y                    0
cluster                   0
dtype: int64

In [22]:
train_all['year'] = train_all['date'].dt.year
train_all['weekday'] = train_all['date'].dt.dayofweek
train_all['month'] = train_all['date'].dt.month
train_all

,id,date,store_nbr,family,sales,onpromotion,type_x,locale,locale_name,description,transferred,dcoilwtico_interpolate,city,state,type_y,cluster,year,weekday,month
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,93.14,Quito,Pichincha,D,13,2013,1,1
1,1,2013-01-01,1,BABY CARE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,93.14,Quito,Pichincha,D,13,2013,1,1
2,2,2013-01-01,1,BEAUTY,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,93.14,Quito,Pichincha,D,13,2013,1,1
3,3,2013-01-01,1,BEVERAGES,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,93.14,Quito,Pichincha,D,13,2013,1,1
4,4,2013-01-01,1,BOOKS,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,93.14,Quito,Pichincha,D,13,2013,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2017,1,8
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2017,1,8
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2017,1,8
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,Quito,Pichincha,B,6,2017,1,8


In [23]:
test_all['year'] = test_all['date'].dt.year
test_all['weekday'] = test_all['date'].dt.dayofweek
test_all['month'] = test_all['date'].dt.month
test_all

,id,date,store_nbr,family,onpromotion,type_x,locale,locale_name,description,transferred,dcoilwtico_interpolate,city,state,type_y,cluster,year,weekday,month
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Normal,Normal,Normal,Normal,False,46.80,Quito,Pichincha,D,13,2017,2,8
1,3000889,2017-08-16,1,BABY CARE,0,Normal,Normal,Normal,Normal,False,46.80,Quito,Pichincha,D,13,2017,2,8
2,3000890,2017-08-16,1,BEAUTY,2,Normal,Normal,Normal,Normal,False,46.80,Quito,Pichincha,D,13,2017,2,8
3,3000891,2017-08-16,1,BEVERAGES,20,Normal,Normal,Normal,Normal,False,46.80,Quito,Pichincha,D,13,2017,2,8
4,3000892,2017-08-16,1,BOOKS,0,Normal,Normal,Normal,Normal,False,46.80,Quito,Pichincha,D,13,2017,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,Normal,Normal,Normal,Normal,False,47.26,Quito,Pichincha,B,6,2017,3,8
28508,3029396,2017-08-31,9,PREPARED FOODS,0,Normal,Normal,Normal,Normal,False,47.26,Quito,Pichincha,B,6,2017,3,8
28509,3029397,2017-08-31,9,PRODUCE,1,Normal,Normal,Normal,Normal,False,47.26,Quito,Pichincha,B,6,2017,3,8
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,Normal,Normal,Normal,Normal,False,47.26,Quito,Pichincha,B,6,2017,3,8


In [24]:
from sklearn.preprocessing import LabelEncoder

train_all = train_all.drop(['id'],axis=1)

for column in train_all:
    if(train_all[column].dtype == object):
        le = LabelEncoder()
        le.fit(train_all[column])
        train_all[column] = le.transform(train_all[column])
        
test_all = test_all.drop(['id'],axis=1)

for column in test_all:
    if(test_all[column].dtype == object):
        le = LabelEncoder()
        le.fit(test_all[column])
        test_all[column] = le.transform(test_all[column])

In [25]:
train_all

,date,store_nbr,family,sales,onpromotion,type_x,locale,locale_name,description,transferred,dcoilwtico_interpolate,city,state,type_y,cluster,year,weekday,month
0,2013-01-01,1,0,0.000,0,3,1,4,51,False,93.14,18,12,3,13,2013,1,1
1,2013-01-01,1,1,0.000,0,3,1,4,51,False,93.14,18,12,3,13,2013,1,1
2,2013-01-01,1,2,0.000,0,3,1,4,51,False,93.14,18,12,3,13,2013,1,1
3,2013-01-01,1,3,0.000,0,3,1,4,51,False,93.14,18,12,3,13,2013,1,1
4,2013-01-01,1,4,0.000,0,3,1,4,51,False,93.14,18,12,3,13,2013,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,2017-08-15,9,28,438.133,0,3,0,20,28,False,47.57,18,12,1,6,2017,1,8
3054344,2017-08-15,9,29,154.553,1,3,0,20,28,False,47.57,18,12,1,6,2017,1,8
3054345,2017-08-15,9,30,2419.729,148,3,0,20,28,False,47.57,18,12,1,6,2017,1,8
3054346,2017-08-15,9,31,121.000,8,3,0,20,28,False,47.57,18,12,1,6,2017,1,8


In [26]:
test_all

,date,store_nbr,family,onpromotion,type_x,locale,locale_name,description,transferred,dcoilwtico_interpolate,city,state,type_y,cluster,year,weekday,month
0,2017-08-16,1,0,0,1,1,1,1,False,46.80,18,12,3,13,2017,2,8
1,2017-08-16,1,1,0,1,1,1,1,False,46.80,18,12,3,13,2017,2,8
2,2017-08-16,1,2,2,1,1,1,1,False,46.80,18,12,3,13,2017,2,8
3,2017-08-16,1,3,20,1,1,1,1,False,46.80,18,12,3,13,2017,2,8
4,2017-08-16,1,4,0,1,1,1,1,False,46.80,18,12,3,13,2017,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,2017-08-31,9,28,1,1,1,1,1,False,47.26,18,12,1,6,2017,3,8
28508,2017-08-31,9,29,0,1,1,1,1,False,47.26,18,12,1,6,2017,3,8
28509,2017-08-31,9,30,1,1,1,1,1,False,47.26,18,12,1,6,2017,3,8
28510,2017-08-31,9,31,9,1,1,1,1,False,47.26,18,12,1,6,2017,3,8


In [27]:
train_all_xt = train_all[train_all['date'] <= '2016-12-31'].drop(['sales','date'], axis=1)
train_all_yt = train_all.loc[train_all['date'] <= '2016-12-31','sales']

train_all_xv = train_all[train_all['date'] > '2016-12-31'].drop(['sales','date'], axis=1)
train_all_yv = train_all.loc[train_all['date'] > '2016-12-31','sales']

In [28]:
train_all_xt = train_all_xt.drop(['locale_name','description'], axis=1)
train_all_xv = train_all_xv.drop(['locale_name','description'], axis=1)

In [29]:
# xt = x train
# yt = y train

# yt = f(xt) 

# xv = x valid
# yv = y valid

In [30]:
train_all_xv

,store_nbr,family,onpromotion,type_x,locale,transferred,dcoilwtico_interpolate,city,state,type_y,cluster,year,weekday,month
2642706,1,0,0,3,1,True,53.055,18,12,3,13,2017,6,1
2642707,1,1,0,3,1,True,53.055,18,12,3,13,2017,6,1
2642708,1,2,0,3,1,True,53.055,18,12,3,13,2017,6,1
2642709,1,3,0,3,1,True,53.055,18,12,3,13,2017,6,1
2642710,1,4,0,3,1,True,53.055,18,12,3,13,2017,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,9,28,0,3,0,False,47.570,18,12,1,6,2017,1,8
3054344,9,29,1,3,0,False,47.570,18,12,1,6,2017,1,8
3054345,9,30,148,3,0,False,47.570,18,12,1,6,2017,1,8
3054346,9,31,8,3,0,False,47.570,18,12,1,6,2017,1,8


In [31]:
test_all = test_all.drop(['date','locale_name','description'], axis=1)
test_all

,store_nbr,family,onpromotion,type_x,locale,transferred,dcoilwtico_interpolate,city,state,type_y,cluster,year,weekday,month
0,1,0,0,1,1,False,46.80,18,12,3,13,2017,2,8
1,1,1,0,1,1,False,46.80,18,12,3,13,2017,2,8
2,1,2,2,1,1,False,46.80,18,12,3,13,2017,2,8
3,1,3,20,1,1,False,46.80,18,12,3,13,2017,2,8
4,1,4,0,1,1,False,46.80,18,12,3,13,2017,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,9,28,1,1,1,False,47.26,18,12,1,6,2017,3,8
28508,9,29,0,1,1,False,47.26,18,12,1,6,2017,3,8
28509,9,30,1,1,1,False,47.26,18,12,1,6,2017,3,8
28510,9,31,9,1,1,False,47.26,18,12,1,6,2017,3,8


In [32]:
train_all_yt.head(3)

0    0.0
1    0.0
2    0.0
Name: sales, dtype: float64

In [33]:
from sklearn.metrics import mean_squared_error
def rmsle(act, pred):
    value = np.sqrt(mean_squared_error(np.log(act+1), np.log(pred+1)))
    return value

from lightgbm import LGBMRegressor
import lightgbm as lgb

model_lgb = LGBMRegressor(objective='regression')

In [34]:
train_all_ytl = np.log(train_all_yt+1)
model_lgb.fit(train_all_xt,train_all_ytl)

LGBMRegressor(objective='regression')

In [43]:
print(train_all_ytl.head(60))
print(train_all_yt.head(60))

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    0.0
31    0.0
32    0.0
33    0.0
34    0.0
35    0.0
36    0.0
37    0.0
38    0.0
39    0.0
40    0.0
41    0.0
42    0.0
43    0.0
44    0.0
45    0.0
46    0.0
47    0.0
48    0.0
49    0.0
50    0.0
51    0.0
52    0.0
53    0.0
54    0.0
55    0.0
56    0.0
57    0.0
58    0.0
59    0.0
Name: sales, dtype: float64
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    0.0
31    0.0
32    0.0
33    0.0
34    0.0
35    0.0
36    0.0
37

In [35]:
train_all_xv

,store_nbr,family,onpromotion,type_x,locale,transferred,dcoilwtico_interpolate,city,state,type_y,cluster,year,weekday,month
2642706,1,0,0,3,1,True,53.055,18,12,3,13,2017,6,1
2642707,1,1,0,3,1,True,53.055,18,12,3,13,2017,6,1
2642708,1,2,0,3,1,True,53.055,18,12,3,13,2017,6,1
2642709,1,3,0,3,1,True,53.055,18,12,3,13,2017,6,1
2642710,1,4,0,3,1,True,53.055,18,12,3,13,2017,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,9,28,0,3,0,False,47.570,18,12,1,6,2017,1,8
3054344,9,29,1,3,0,False,47.570,18,12,1,6,2017,1,8
3054345,9,30,148,3,0,False,47.570,18,12,1,6,2017,1,8
3054346,9,31,8,3,0,False,47.570,18,12,1,6,2017,1,8


In [36]:
pred_log = model_lgb.predict(train_all_xv)
pred = np.exp(pred_log)-1

In [37]:
rmse = rmsle(train_all_yv,pred)
rmse

0.7915445744337347

In [38]:
result = model_lgb.predict(test_all)

In [39]:
output = pd.DataFrame({'id': test["id"], 'sales': result})
output

,id,sales
0,3000888,1.634943
1,3000889,0.296509
2,3000890,2.865074
3,3000891,7.910692
4,3000892,0.355660
...,...,...
28507,3029395,5.494931
28508,3029396,4.630773
28509,3029397,7.018904
28510,3029398,3.378844


In [40]:
output.to_csv('submission.csv', index=False)